In [1]:
# -*- coding: utf-8 -*-
# -*- authors : Vincent Roduit -*-
# -*- date : 2025-04-24 -*-
# -*- Last revision: 2025-04-09 by roduit -*-
# -*- python version : 3.11.11 -*-
# -*- Description: Notebook that summarizeses the main results-*-

# <center> EE-452: Network Machine Learning </center>
## <center> Ecole Polytechnique Fédérale de Lausanne </center>
### <center>Graph-based EEG Analysis </center>
---

In [19]:
#import libraries
import pandas as pd
from seiz_eeg.dataset import EEGDataset
from torch.utils.data import DataLoader

#import modules
import constants
from processing import *
from cnn_base import CnnModel
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Data

## Train

In [23]:
clips_train = pd.read_parquet(constants.CLIPS_TRAIN_DIR)

dataset_tr = EEGDataset(
    clips_train,
    signals_root=constants.TRAIN_DIR,
    signal_transform=fft_filtering,
    prefetch=True)

loader_train = DataLoader(dataset_tr, batch_size=constants.BATCH_SIZE, shuffle=True)


## Test

In [24]:
clips_test = pd.read_parquet(constants.CLIPS_TEST_DIR)

dataset_te = EEGDataset(
    clips_test,
    signals_root=constants.TEST_DIR,
    signal_transform=fft_filtering,
    prefetch=True)
loader_te = DataLoader(dataset_te, batch_size=constants.BATCH_SIZE, shuffle=False)

# Models

## Basic CNN

In [40]:
basic_cnn = CnnModel(input_shape=19, output_shape=1)

In [41]:
basic_cnn.train_model(loader_tr=loader_train, num_epochs=1)

Training: 100%|██████████| 1/1 [00:08<00:00,  8.15s/it]
